In [1]:
import polars as pl

In [2]:
grades = pl.DataFrame({
    'student': ['bas', 'laura', 'tim', 'jenny'],
    'math': [10, 5, 6, 8],
    'biology': [4, 6, 2, 7],
    'geography': [8, 4, 9, 7]})
grades

student,math,biology,geography
str,i64,i64,i64
"""bas""",10,4,8
"""laura""",5,6,4
"""tim""",6,2,9
"""jenny""",8,7,7


In [3]:
grades.select([
    pl.concat_list(pl.all().exclude('student')).alias('grades')])

grades
list[i64]
"[10, 4, 8]"
"[5, 6, 4]"
"[6, 2, 9]"
"[8, 7, 7]"


In [4]:
rank_pct = pl.element().rank(descending=True) / pl.col('').count()

In [7]:
grades.with_columns(
    pl.concat_list(pl.all().exclude('student'))
        .alias('grades')
).select([
    pl.all().exclude('grades'),
    pl.col('grades')
        .arr
        .eval(rank_pct, parallel=False)
        .alias('grade_rank')])

student,math,biology,geography,grade_rank
str,i64,i64,i64,list[f64]
"""bas""",10,4,8,"[0.333333, 1.0, 0.666667]"
"""laura""",5,6,4,"[0.666667, 0.333333, 1.0]"
"""tim""",6,2,9,"[0.666667, 1.0, 0.333333]"
"""jenny""",8,7,7,"[0.333333, 0.833333, 0.833333]"
